In [4]:
"""
Use of openai plus langchain for processing information in a pdf
Generated using chatGPT for incorporating asyncio for concurrent running of prompts
Generated by pasting my code from the analysis_v3 script with the following question:
Can you modify the below python code to incorporate asyncio to allow concurrent running of the paper_search() function?
"""
import sys 
import requests
import os
from pathlib import Path  # directory setting
import asyncio # For async asking of prompts
import json
from langchain.docstore.document import Document

import httpx  # for elsevier and semantic scholar api calls
from habanero import Crossref, cn  # Crossref database accessing
from dotenv import load_dotenv, find_dotenv  # loading in API keys
from langchain.document_loaders import PyPDFLoader # document loader import
from langchain.chat_models import ChatOpenAI  # LLM import
from langchain import LLMChain  # Agent import
from langchain.prompts.chat import ( # prompts for designing inputs
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
import openai
from langchain.indexes import VectorstoreIndexCreator
import fitz #pdf reading library
import json
from pyalex import Works #, Authors, Sources, Institutions, Concepts, Publishers, Funders
import pyalex
import PyPDF2
import io
#import tiktoken
from jsonformer import Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer

from dotenv import load_dotenv
import os

from langchain_huggingface import HuggingFacePipeline
from langchain_experimental.llms import JsonFormer
from transformers import pipeline

# Specify the path to your .env file if it's not in the same directory
dotenv_path = '.env'

# Load environment variables from the .env file
load_dotenv(dotenv_path)

True

In [5]:
"""This function is a stand in. GROBID will eventually replace this and return fully markdown available text for greater functionality. 
For step one, we simply upload a URL from the Nodes Server."""
def get_pdf_text(pdf_CID):
    ipfs="https://ipfs.desci.com/ipfs/"+pdf_CID
  
    response = requests.get(ipfs) 
    
    if response.status_code == 200:
        # Open the PDF content with PyPDF2
        pdf_file = PyPDF2.PdfReader(io.BytesIO(response.content))
        
        # Check if the PDF is extractable
        if pdf_file.is_encrypted:
            pdf_file.decrypt("")  # Assuming the PDF has no password
            
        
        # Initialize text
        pdf_text = ""
        
        # Limit processing to under the relevant token limit
        num_pages = len(pdf_file.pages)
        pdf_word_count = 0

        for i in range(num_pages):
            page = pdf_file.pages[i]
            page_text = page.extract_text()
            
            if page_text is not None:
                page_word_count = len(page_text.split())
                total_word_count = pdf_word_count + page_word_count

                if total_word_count < 16000*.5:
                    pdf_text += page_text
                    pdf_word_count = total_word_count

                else: 
                    break
                 
        if pdf_word_count < 20: 
            print(f"Not enough information in the PDF from {pdf_CID}. This could be because this PDF is rendered as an image, or doesn't have many words.")
            return None
            
        return pdf_text
    else:
        print(f"Error fetching PDF from {pdf_CID}. Status code: {response.status_code}")
        return None

In [8]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

hf_model = pipeline(
    "text-generation", model="cerebras/Cerebras-GPT-590M", max_new_tokens=200
)

original_model = HuggingFacePipeline(pipeline=hf_model)


# Define a simple JSON schema
decoder_schema = {
    "title": "Claims with Context Schema",
    "type": "object",
    "properties": {
        "claims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "claim": {
                        "type": "string",
                        "description": "The claim made in the document."
                    },
                    "context": {
                        "type": "string",
                        "description": "The context or evidence supporting the claim it should answer the questions what, how, or who according to the prompt."
                    }
                },
                "required": ["claim", "context"]
            }
        }
    }
    }


# Create a JSONFormer instance
text = get_pdf_text("bafybeigtdu677sjhof34ad65zxsdndfodmmwrf33ori2xf24llh2vg3fea")
doc_text = Document(page_content = text).page_content


# Define a prompt
prompt = f"""You are a world class research assistant who produces answers based on single research papers. \
                        You are tasked with reading the following publication text and responding to the questions just using a JSON format: \n\n {doc_text}\n\n.\
                        In this case your job is to search through the text and extract the most relevant scientific claims from the text where a scientific claim \
                        is a statement made in science based on an experiment. They're claims just like you might find in other classes, but they are backed by experimental data.In addition you should extract as much relevant context as you can for each claim. The context you should extract should answer the questions of \

                        1. What observable measures/data were collected\
                        2. How (with what methods, analyses, etc.) from\
                        3. Who(m) (which participants, what dataset, what population, etc.)\
                        
                        When you've found all the relevant claims and context in the text say.\

                        Example: \
                        
                        Human: "please extract all the claims from this research paper" \
                        AI Assistent: \
                        "claim": "US patents filed by inventors who were new to the patent's field tended to be more novel",\
                        "context": \
                            "To assess patent novelty, we calculate new combinations (ln) as the logarithmic transformation of one plus the number of pairwise subclass combinations of a patent that appear for the first time in the US. patent database (Fleming et al. 2007, Jung and Jeongsik 2016). To do so, each pairwise combination of subclasses is compared with all pairwise combinations of all prior U.S. patents. (p. 5)",\
                            "we begin with the full population of inventors and collect all patents assigned to \ufb01rms but, by design, must restrict the sample to inventors who have at least two patents assigned to the same \ufb01rm. The advantage of this panel setup is that we can use inventor\u2013firm fixed effect models to control for unobserved heterogeneity among inventors and firms, which arguably have a strong effect on the novelty and value of creative output. This approach basically uses repeated patents of the same inventor within the same firm to identify whether the inventor creates more or less novel\u2014and more or less valuable\u2014patents when any subsequent patent is categorized in a new \ufb01eld. The sample includes 2,705,431 patent\u2013inventor observations assigned to 396,336 unique inventors and 46,880 unique firms, accounting for 473,419 unique inventor\u2013firm pairs. (p. 5)",\
                            "For each inventor-patent observation, we retrieve the three-digit technology classes of all prior patents of the focal inventor and identify whether there is any overlap between the three-digit technology classes of the focal patent and the three-digit technology classes linked o all prior patents of the same inventor. We rely on all classes assigned to a patent rather than just the primary class. Exploring new fields is a binary indicator that equals one in the absence of any overlapping class between all prior patents and the focal patent. (p. 6)",\
                            "we can use inventor\u2013\ufb01rm \ufb01xed effect models to control for unobserved heterogeneity among inventors and \ufb01rms, which arguably have a strong effect on the novelty and value of creative output (p. 5)",\
                            "we select the full population of inventors with U.S. patents assigned to \ufb01rms for 1975\u20132002 (p. 3)"  \
                        """
print(f"Input length: {len(prompt.split())}")  # Check the number of tokens

decoder_schema = {
    "title": "Claims with Context Schema",
    "type": "object",
    "properties": {
        "claims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "claim": {
                        "type": "string",
                        "description": "The claim made in the document."
                    },
                    "context": {
                        "type": "string",
                        "description": "The context or evidence supporting the claim."
                    }
                },
                "required": ["claim", "context"]
            }
        }
    }
}

# Make a prediction
#results = json_former.predict(prompt)
output = original_model.predict(prompt)
print(output)
json_former = JsonFormer(json_schema=decoder_schema, pipeline=hf_model)


results = json_former.predict(prompt)


# Print the results
print(results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input length: 7098


IndexError: index out of range in self

In [13]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

# Initialize the Hugging Face model
hf_model = pipeline("text-generation", model="databricks/dolly-v2-12b", trust_remote_code="True")
huggingface_model = HuggingFacePipeline(pipeline=hf_model)

# Define the decoder schema
decoder_schema = {
    "title": "Claims with Context Schema",
    "type": "object",
    "properties": {
        "claims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "claim": {
                        "type": "string",
                        "description": "The claim made in the document."
                    },
                    "context": {
                        "type": "string",
                        "description": "The context or evidence supporting the claim."
                    }
                },
                "required": ["claim", "context"]
            }
        }
    }
}

# Function to extract claims and context
def extract_claims_and_context(text):
    prompt = f"Extract claims and their context from the following text:\n{text}\n\nReturn the results in the following JSON format:\n{decoder_schema}"
    response = huggingface_model.predict(prompt)
    return response  # Adjust based on your model's output

text = get_pdf_text("bafybeigtdu677sjhof34ad65zxsdndfodmmwrf33ori2xf24llh2vg3fea")
doc_text = Document(page_content = text).page_content

# Example usage
result = extract_claims_and_context(doc_text)
print(result)

/Users/desot1/Dev/automating-metadata/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


: 

In [ ]:
def langchain_paper_search(pdf_CID):
    #file_path
    """
    Analyzes a pdf document defined by file_path and asks questions regarding the text
    using LLM's.
    The results are returned as unstructured text in a dictionary.
    """
    #%% Setup, defining framework of paper info extraction
    # Define language model to use
    #llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
    hf_model = pipeline("text-generation", model="cerebras/Cerebras-GPT-590M", max_new_tokens=200)

    #%% Extracting info from paper
    # Define the PDF document, load it in
    text = get_pdf_text(pdf_CID)
    doc_text = Document(page_content = text).page_content
    # Defining system and human prompts with variables
    prompt = f"""You are a world class research assistant who produces answers based on single research papers. 
                        You are tasked with reading the following publication text and responding to the questions just using a JSON format: \n\n {doc_text}\n\n.
                        In this case your job is to search through the text and extract the most relevant scientific claims from the text where a scientific claim 
                        is a statement made in science based on an experiment. They're claims just like you might find in other classes, but they are backed by experimental data.In addition you should extract as much relevant context as you can for each claim. The context you should extract should answer the questions of 

                        1. What observable measures/data were collected
                        2. How (with what methods, analyses, etc.) from
                        3. Who(m) (which participants, what dataset, what population, etc.)
                        
                        When you've found all the relevant claims and context in the text say.

                        Example: 
                        
                        Human: "please extract all the claims from this research paper" 
                        AI Assistent: 
                        "claim": "US patents filed by inventors who were new to the patent's field tended to be more novel",
                        "context": 
                            "To assess patent novelty, we calculate new combinations (ln) as the logarithmic transformation of one plus the number of pairwise subclass combinations of a patent that appear for the first time in the US. patent database (Fleming et al. 2007, Jung and Jeongsik 2016). To do so, each pairwise combination of subclasses is compared with all pairwise combinations of all prior U.S. patents. (p. 5)",
                            "we begin with the full population of inventors and collect all patents assigned to \ufb01rms but, by design, must restrict the sample to inventors who have at least two patents assigned to the same \ufb01rm. The advantage of this panel setup is that we can use inventor\u2013firm fixed effect models to control for unobserved heterogeneity among inventors and firms, which arguably have a strong effect on the novelty and value of creative output. This approach basically uses repeated patents of the same inventor within the same firm to identify whether the inventor creates more or less novel\u2014and more or less valuable\u2014patents when any subsequent patent is categorized in a new \ufb01eld. The sample includes 2,705,431 patent\u2013inventor observations assigned to 396,336 unique inventors and 46,880 unique firms, accounting for 473,419 unique inventor\u2013firm pairs. (p. 5)",
                            "For each inventor-patent observation, we retrieve the three-digit technology classes of all prior patents of the focal inventor and identify whether there is any overlap between the three-digit technology classes of the focal patent and the three-digit technology classes linked o all prior patents of the same inventor. We rely on all classes assigned to a patent rather than just the primary class. Exploring new fields is a binary indicator that equals one in the absence of any overlapping class between all prior patents and the focal patent. (p. 6)",
                            "we can use inventor\u2013\ufb01rm \ufb01xed effect models to control for unobserved heterogeneity among inventors and \ufb01rms, which arguably have a strong effect on the novelty and value of creative output (p. 5)",
                            "we select the full population of inventors with U.S. patents assigned to \ufb01rms for 1975\u20132002 (p. 3)"  
                        """

    



  
    # Define the JSON Schema and corresponding schemas in a list
    
    decoder_schema = {
    "title": "Claims with Context Schema",
    "type": "object",
    "properties": {
        "claims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "claim": {
                        "type": "string",
                        "description": "The claim made in the document."
                    },
                    "context": {
                        "type": "string",
                        "description": "The context or evidence supporting the claim it should answer the questions what, how, or who according to the prompt."
                    }
                },
                "required": ["claim", "context"]
            }
        }
    }
    }

    json_former = JsonFormer(json_schema=decoder_schema, pipeline=hf_model)
    results = json_former.predict(prompt, stop=["claim"])
    return results

In [ ]:
output = langchain_paper_search("bafybeigtdu677sjhof34ad65zxsdndfodmmwrf33ori2xf24llh2vg3fea")

print(output)

In [ ]:
def create_discourse(authors): 
    claims = {}  # Dictionary to store author information
    
    for claim in claims: 
    
      claim_info = {
          {
          "id": "megacoglab-W3sdOb60i", #assign in nodes itself. 
          "claim": "US patents filed by inventors who were new to the patent's field tended to be more novel", #extracted
          "citekey": "artsParadiseNoveltyLoss2018a", #pdf CID
          "dataset": "megacoglab", #community? Smthn like this - can we figure this out? 
          "context": [
                  "To assess patent novelty, we calculate new combinations (ln) as the logarithmic transformation of one plus the number of pairwise subclass combinations of a patent that appear for the first time in the US. patent database (Fleming et al. 2007, Jung and Jeongsik 2016). To do so, each pairwise combination of subclasses is compared with all pairwise combinations of all prior U.S. patents. (p. 5)",
                  "we begin with the full population of inventors and collect all patents assigned to \ufb01rms but, by design, must restrict the sample to inventors who have at least two patents assigned to the same \ufb01rm. The advantage of this panel setup is that we can use inventor\u2013firm fixed effect models to control for unobserved heterogeneity among inventors and firms, which arguably have a strong effect on the novelty and value of creative output. This approach basically uses repeated patents of the same inventor within the same firm to identify whether the inventor creates more or less novel\u2014and more or less valuable\u2014patents when any subsequent patent is categorized in a new \ufb01eld. The sample includes 2,705,431 patent\u2013inventor observations assigned to 396,336 unique inventors and 46,880 unique firms, accounting for 473,419 unique inventor\u2013firm pairs. (p. 5)",
                  "For each inventor-patent observation, we retrieve the three-digit technology classes of all prior patents of the focal inventor and identify whether there is any overlap between the three-digit technology classes of the focal patent and the three-digit technology classes linked o all prior patents of the same inventor. We rely on all classes assigned to a patent rather than just the primary class. Exploring new fields is a binary indicator that equals one in the absence of any overlapping class between all prior patents and the focal patent. (p. 6)",
                  "we can use inventor\u2013\ufb01rm \ufb01xed effect models to control for unobserved heterogeneity among inventors and \ufb01rms, which arguably have a strong effect on the novelty and value of creative output (p. 5)",
                  "we select the full population of inventors with U.S. patents assigned to \ufb01rms for 1975\u20132002 (p. 3)"
    ]
  },}
        